In [1]:
import numpy as np
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report, roc_auc_score, average_precision_score
from xgboost import XGBClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import LinearSVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.externals import joblib
import collections
import math
import sys
import time

In [2]:
ml_type = 1

threads_f = 'threads/threads_{}'.format(ml_type)
threads_f_name = '{}.j'.format(threads_f)
threads72_f_name = '{}_72.j'.format(threads_f)
threads729_f_name = '{}_729.j'.format(threads_f)
threads = joblib.load(threads_f_name)
len_threads = len(threads[0]) + len(threads[1])
print('threads', len_threads)

way_dict = joblib.load('border_vars/way_dict.j')
X = joblib.load('border_vars/X.j')
Y = joblib.load('border_vars/Y.j')

print(X.shape, Y.shape)
print('all', dict(collections.Counter(Y)))

x_train, y_train = [], []
model = ''
def gen_train():
    global x_train
    global y_train
    x_train, y_train = [], []
    for i in threads:
        for thread in threads[i]:
            x_train.append(thread)
            y_train.append(i)
    x_train, y_train = np.array(x_train), np.array(y_train)

threads 14
(7290000, 8) (7290000,)
all {1: 6661743, 0: 628257}


In [5]:
%%time
print('#', len_threads)
gen_train()

if ml_type == 1:
    model = XGBClassifier()
elif ml_type == 2:
    model = LogisticRegression()
elif ml_type == 3:
    model = LinearSVC(random_state=0, tol=1e-5)
elif ml_type == 4:
    model = KNeighborsClassifier(n_neighbors=6)

model.fit(x_train, y_train)
y_pred = model.predict(X)
accuracy = accuracy_score(Y, y_pred)
print('Accuracy: {}'.format(accuracy))
#print(y_pred)
print(dict(collections.Counter(y_pred)))

# 14
Accuracy: 0.6762426611796982
{1: 4383720, 0: 2906280}
CPU times: user 8.22 s, sys: 1.98 s, total: 10.2 s
Wall time: 10.7 s


In [6]:
%%time
find = 1 if len(threads[0]) > len(threads[1]) else 0

def distance(l0):
    return min([math.sqrt(sum(map(lambda x: (x[0] - x[1])**2, zip(thread, l0)))) for thread in threads[find]])

t_id = 0
max_distance = 0
max_dis_thread = 0
for i in range(len(y_pred)):
    if y_pred[i] == 0:
        for j in way_dict[i]:
            if y_pred[j] == 1:
                t_id = j if find else i
                dis = distance(list(X[t_id]))
                if dis > max_distance:
                    max_distance = dis
                    max_dis_thread = t_id

print([max_distance, max_dis_thread])
print(Y[max_dis_thread])

[14.2828568570857, 5102990]
1
CPU times: user 1min 10s, sys: 2.2 s, total: 1min 12s
Wall time: 1min 13s


In [16]:
%%time
find = 1 if len(threads[0]) > len(threads[1]) else 0

def distance(l0):
    #return min([math.sqrt(sum(map(lambda x: (x[0] - x[1])**2, zip(thread, l0)))) for thread in threads[find]])
    _min = 99999
    for thread in threads[find]:
        _sum = 0
        for i in range(len(l0)):
            _sum += (thread[i] - l0[i])**2
        dis = math.sqrt(_sum)
        if dis < _min:
            _min = dis
    return _min
            

def if_distance(tid):
    global max_distance
    global max_dis_thread
    dis = distance(list(X[tid]))
    if dis > max_distance:
        max_distance = dis
        max_dis_thread = tid

t_id = 0
max_distance = 0
max_dis_thread = 0
#[if_distance(j if find else i) for i in range(len(y_pred)) if y_pred[i] == 0 for j in way_dict[i] if y_pred[j] == 1]

for i in range(len(y_pred)):
    if y_pred[i] == 0:
        for j in way_dict[i]:
            if y_pred[j] == 1:
                if_distance(j if find else i)
                
print([max_distance, max_dis_thread])
print(Y[max_dis_thread])

[14.2828568570857, 5102990]
1
CPU times: user 1min 4s, sys: 324 ms, total: 1min 4s
Wall time: 1min 5s


In [18]:
import multiprocessing



if __name__ == "__main__": 
    q_x = multiprocessing.Queue()
    q_min = multiprocessing.Queue()
    x_min_id = multiprocessing.Value('i')
    x_min_dis = multiprocessing.Value('d')
    q_x.put([1,2,3,4])
    print(q_x.get(), q_x.empty())
    
    

[1, 2, 3, 4] True
